<a href="https://colab.research.google.com/github/scaairesearch/da_demo/blob/main/gradio_da_demo_mnist_mnist_m_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio --quiet

In [5]:
import gradio as gr
import os
from PIL import Image
from torchvision import datasets,transforms
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from collections import OrderedDict
import pandas as pd
import io
import base64

In [6]:
def display_image():
    # Load the image from a local file
    image = Image.open("/content/mnist-m.JPG")
    return image

In [7]:
img_size = 28 # for mnist
cpu_batch_size = 10
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# target_dataset_name = 'mnist_m'
# target_image_root = os.path.join(data_base_path, target_dataset_name)
# cuda_batch_size = 128
# EPOCHS = 1 # 15 if cuda else 2

# target_drive_path = '/content/gdrive/MyDrive/mnist_m/dataset/mnist_m.tar.gz'
# model_drive_path = '/content/gdrive/MyDrive/da_demo/cv/models'
# demo_data_path = '/content/gdrive/MyDrive/da_demo/data'

# data_base_path = '/content/data'
# source_dataset_name = 'MNIST'
# target_dataset_name = 'mnist_m'
# source_image_root = os.path.join(data_base_path, source_dataset_name)
# target_image_root = os.path.join(data_base_path, target_dataset_name)
# model_root = 'models'
# cuda = True
# cudnn.benchmark = True
# lr = 1e-3
# batch_size = 128
# image_size = 28
# n_epoch = 100

### SOURCE


In [8]:
# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize(img_size),
                                       transforms.ToTensor(),# converts to tesnor
                                      #  transforms.Normalize((0.1307,), (0.3081,))
                                       ])

# display_transforms = transforms.Compose([
#                                        transforms.Resize(img_size),
#                                       #  transforms.ToTensor(),# converts to tesnor
#                                       #  transforms.Normalize((0.1307,), (0.3081,)),
#                                        transforms.ToPILImage()
#                                        ])

transform_to_pil  = transforms.ToPILImage()

test = datasets.MNIST('./data',
                      train=False,
                      download=True,
                      transform=test_transforms)

dataloader_args = dict(shuffle=True, batch_size=cpu_batch_size)

mnist_loader = torch.utils.data.DataLoader(
    dataset = test,
    **dataloader_args
)

def get_mnist_images():
    images, labels = next(iter(mnist_loader))
    pil_images = [transform_to_pil(image) for image in images]
    return pil_images

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 51166894.44it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1676154.61it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 12468321.92it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7823625.78it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## TARGET

In [9]:
splits = {'train': 'data/train-00000-of-00001-571b6b1e2c195186.parquet', 'test': 'data/test-00000-of-00001-ba3ad971b105ff65.parquet'}
df = pd.read_parquet("hf://datasets/Mike0307/MNIST-M/" + splits["test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
# df['image'].iloc[0]

In [38]:
class MNIST_M(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=None):
      # splits = {'train': 'data/train-00000-of-00001-571b6b1e2c195186.parquet', 'test': 'data/test-00000-of-00001-ba3ad971b105ff65.parquet'}
      # self.dataframe = pd.read_parquet("hf://datasets/Mike0307/MNIST-M/" + splits["test"])
      self.dataframe = dataframe
      self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get image and label from dataframe
        img_data = self.dataframe.iloc[idx]['image']['bytes']
        label = self.dataframe.iloc[idx]['label']
        img_path = self.dataframe.iloc[idx]['image']['path']

        # Decode image data (assuming it's base64 encoded)
        img = Image.open(io.BytesIO(img_data))

        # img = Image.open(io.BytesIO(base64.b64decode(img_data)))

        # Apply transformations if any
        if self.transform:
            img = self.transform(img)

        return img, label,img_path


# Test Phase transformations
target_test_transforms = transforms.Compose([
                                       transforms.Resize(img_size),
                                       transforms.ToTensor(),# converts to tesnor
                                       transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                                       ])

# transform_to_pil  = transforms.Compose([
#                                        transforms.Resize(img_size),
#                                        transforms.ToTensor(),# converts to tesnor
#                                        transforms.ToPILImage()
#                                        ])

transform_to_pil  = transforms.ToPILImage()



# Create dataset
target_test_dataset = MNIST_M(dataframe=df, transform=target_test_transforms)
target_test_dataloader = torch.utils.data.DataLoader(target_test_dataset, batch_size=cpu_batch_size, shuffle=True)
def get_mnist_m_images():
    images, labels,image_names = next(iter(target_test_dataloader))
    pil_images = [transform_to_pil(image) for image in images]
    return pil_images, labels.tolist()

# Load Models

In [13]:
def get_device():
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
    print("Device Selected:", device)
    return device

device = get_device()
print(device)

Device Selected: cpu
cpu


In [14]:

class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

class Network(nn.Module):
    def __init__(self, num_classes = 10):
        super(Network, self).__init__()  # Initialize the parent class

        drop_out_value = 0.1

        #---------------------Feature Extractor Network------------------------#
        self.feature_extractor  = nn.Sequential(
            # Input Block
            nn.Conv2d(3, 16, 3, bias=False),  # In: 3x28x28, Out: 16x26x26, RF: 3x3, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(drop_out_value),

            # Conv Block 2
            nn.Conv2d(16, 16, 3, bias=False),  # In: 16x26x26, Out: 16x24x24, RF: 5x5, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(drop_out_value),

            # Conv Block 3
            nn.Conv2d(16, 16, 3, bias=False),  # In: 16x24x24, Out: 16x22x22, RF: 7x7, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(drop_out_value),

            # Transition Block 1
            nn.MaxPool2d(kernel_size=2, stride=2),  # In: 16x22x22, Out: 16x11x11, RF: 8x8, Stride: 2

            # Conv Block 4
            nn.Conv2d(16, 16, 3, bias=False),  # In: 16x11x11, Out: 16x9x9, RF: 12x12, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(drop_out_value),

            # Conv Block 5
            nn.Conv2d(16, 32, 3, bias=False),  # In: 16x9x9, Out: 32x7x7, RF: 16x16, Stride: 1
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(drop_out_value),

            # Output Block
            nn.Conv2d(32, 64, 1, bias=False),  # In: 32x7x7, Out: 64x7x7, RF: 16x16, Stride: 1

            # Global Average Pooling
            nn.AvgPool2d(7)  # In: 64x7x7, Out: 64x1x1, RF: 16x16, Stride: 7
        )

        #---------------------Class Classifier Network------------------------#
        self.class_classifier = nn.Sequential(nn.ReLU(),
                                        nn.Dropout(p=drop_out_value),
                                        nn.Linear(64,50),
                                        nn.BatchNorm1d(50), # added batch norm to improve accuracy
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_out_value),
                                        nn.Linear(50,num_classes))

        #---------------------Label Classifier Network------------------------#
        self.domain_classifier = nn.Sequential(nn.ReLU(),
                                        nn.Dropout(p=drop_out_value),
                                        nn.Linear(64,50),
                                        nn.BatchNorm1d(50), # added batch norm to improve accuracy
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_out_value),
                                        nn.Linear(50,2))
    def forward(self, input_data, alpha = 1.0):
      if input_data.data.shape[1] == 1:
        input_data = input_data.expand(input_data.data.shape[0], 3, img_size, img_size)

      input_data = self.feature_extractor(input_data)

      features = input_data.view(input_data.size(0), -1)  # Flatten the output for fully connected layer

      reverse_features = GradientReversalFn.apply(features, alpha)
      class_output = self.class_classifier(features)
      domain_output = self.domain_classifier(reverse_features)

      return class_output, domain_output, features

In [16]:
## NON DANN
# Instantiate the model (make sure it has the same architecture)
loaded_model_non_dann = Network()
loaded_model_non_dann = loaded_model_non_dann.to(device)
# Load the saved state dictionary
loaded_model_non_dann.load_state_dict(torch.load('/content/non_dann_26_06.pt', map_location=device), strict=False)
loaded_model_non_dann.eval()

Network(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (5): ReLU()
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.1, inplace=False)
    (8): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (9): ReLU()
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.1, inplace=False)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (14): ReLU()
    (15): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (16): Dropout(p=0.1, inplace=False)
    (1

In [17]:
##  DANN
# Instantiate the model (make sure it has the same architecture)
loaded_model_dann = Network()
loaded_model_dann = loaded_model_dann.to(device)
# Load the saved state dictionary
loaded_model_dann.load_state_dict(torch.load('/content/dann_26_06.pt', map_location=device), strict=False)
loaded_model_dann.eval()

Network(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.1, inplace=False)
    (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (5): ReLU()
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.1, inplace=False)
    (8): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (9): ReLU()
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.1, inplace=False)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (14): ReLU()
    (15): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (16): Dropout(p=0.1, inplace=False)
    (1

# Classify Image

In [57]:
def classify_image(image,
                   select_model
                  #  ,mnist_images, mnist_m_images
                   ):
  source = "Unknown"
  if image in mnist_images:
      source = "MNIST"
  elif image in mnist_m_images:
      source = "MNIST-M"



  if select_model == "Baseline (Non-DANN)":
    model = loaded_model_non_dann
    test_transforms = transforms.Compose([
                                       transforms.Resize(img_size),
                                       transforms.ToTensor(),# converts to tesnor
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])
    transformed_image = test_transforms(image)
    image_tensor = transformed_image.to(device).unsqueeze(0)
  if select_model == "DANN":
    ## TO DO Transforms for DANN
    model = loaded_model_dann
    target_test_transforms = transforms.Compose([
                                       transforms.Resize(img_size),
                                       transforms.ToTensor(),# converts to tesnor
                                       transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                                       ])
    target_transformed_image = target_test_transforms(image)
    image_tensor = target_transformed_image.to(device).unsqueeze(0)


  logits,_,_ = model(image_tensor)
  output = F.softmax(logits.view(-1), dim = -1) #F.softmax(output.flatten(), dim=-1) #

  confidences = [(class_names[i], float(output[i])) for i in range(len(class_names))]
  confidences.sort(key=lambda x: x[1], reverse=True)
  confidences = OrderedDict(confidences[:3])
  label = torch.argmax(output).item()


  # output =  torch.argmax(F.softmax(class_output.view(-1))).item()
  # label_output = class_output.argmax(dim=1)#.cpu().numpy()
  # label_output = torch.argmax(label_output).item()

  return confidences#, source
  # return f'{select_model} {label_output} {output}'

In [19]:
# Function to set the source of the selected image
def set_image_source(image, source, state):
    state['source'] = source
    return state['source']

In [37]:
# # Define the interaction for updating ground truth label
# def update_ground_truth(image):
#     idx = mnist_images.index(image)
#     return mnist_labels[idx]

# Version 4 - Case 1

MNIST M images not classified correctly with NON DANN but classified correctly with DANN

In [76]:
case1_list= torch.load('/content/list_mnist_m_non_dann_misclassified_dann_classified.pt')

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [74]:
with gr.Blocks() as demo:
  with gr.Tab("Case 1: MNIST_M_Non_DANN_Misclassify_DANN_Classify"):
    gr.Markdown(value = "XXX")


demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0f1aae1f7e2954721a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://0f1aae1f7e2954721a.gradio.live


In [1]:
    mnist_images = get_mnist_images()
    mnist_m_images,mnist_m_labels = get_mnist_m_images()

NameError: name 'get_mnist_images' is not defined

# Version 3 - Working - Inference and Introduction

In [73]:

### NEW VERSION V3

with gr.Blocks() as demo:

  with gr.Tab("MNIST-M : inference NonDANN and DANN"):
    with gr.Row():
      radio_model = gr.Radio(["Baseline (Non-DANN)", "DANN"],
                               label="Select the model you want to use.",
                               value="Baseline (Non-DANN)",  # Set default value
                               scale=2)
    # with gr.Row():
    #   radio_dataset = gr.Radio(["MNIST", "MNIST-M"],
    #                            label="Select the dataset you want to use.",
    #                            value="MNIST",  # Set default value
    #                            scale=2)


    with gr.Row():
      with gr.Column():
        input_image_classify = gr.Image(label="Classify Digit", type = "pil", height = 256, width = 256)
        button_classify = gr.Button("Submit to Classify Image", visible = True, size ='sm')


      with gr.Column():
        with gr.Row():
          label_classify = gr.Label(label = "Predicted label", num_top_classes=10, visible = True)

        # with gr.Row():
        #   label_gt = gr.Label(label = "Ground Truth label", visible = True)

    # # State to store the source of the selected image
    # state = gr.State({"source": None})




    with gr.Row():
      gr.Examples(mnist_images,inputs=[input_image_classify], label = "Select an example MNIST Image")


    with gr.Row():
      gr.Examples(mnist_m_images,inputs=[input_image_classify], label = "Select an example MNIST-M Image") #working

    # with gr.Row():
    #   examples = gr.Examples(
    #       examples=[[img] for img in mnist_m_images],
    #       inputs=[input_image_classify],
    #       # examples=[[img,label] for img,label in zip(mnist_m_images,mnist_m_labels)],
    #       # inputs=[input_image_classify,label_gt],
    #       label="Select an example MNIST Image")
    with gr.Row():
      gr.Markdown(value = f'MNIST- M Ground Truth Label = {[label for label in mnist_m_labels]}')




  ##########################
  with gr.Tab("Introduction"):
    gr.Markdown("## Domain Adaptation in Deep Networks - Demonstration")
    gr.Markdown(
        '''
        Source - MNIST
        ------
        - The MNIST database (Modified National Institute of Standards and Technology database) is a large collection of handwritten digits.
        - It has a training set of 60,000 examples, and a test set of 10,000 examples.
        - 28 x 28 size
        - 1 channel

        '''
        )
    gr.Markdown(
        '''
        Target - MNIST-M
        -------
        - MNIST-M is created by combining MNIST digits with the patches randomly extracted from color photos of BSDS500 as their background.
        - It contains 59,001 training and 90,001 test images.
        - 28 x 28 size
        - 3 channels
        '''
    )
    with gr.Row():
        with gr.Column():
             image_output = gr.Image(value=display_image(), label = "source and target",height = 512, width = 512, show_label = True)
  ################

  button_classify.click(fn=classify_image,
                        inputs=[input_image_classify,radio_model],
                        outputs=[label_classify])




demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f6659e275a9a19ae0b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://f6659e275a9a19ae0b.gradio.live


# Version 2

In [34]:

### NEW VERSION V2
with gr.Blocks() as demo:

  with gr.Tab("MNIST-M : inference NonDANN and DANN"):
    with gr.Row():
      radio_model = gr.Radio(["Baseline (Non-DANN)", "DANN"],
                               label="Select the model you want to use.",
                               value="Baseline (Non-DANN)",  # Set default value
                               scale=2)
    with gr.Row():
      radio_dataset = gr.Radio(["MNIST", "MNIST-M"],
                               label="Select the dataset you want to use.",
                               value="MNIST",  # Set default value
                               scale=2)


    with gr.Row():
      with gr.Column():
        input_image_classify = gr.Image(label="Classify Digit", type = "pil", height = 256, width = 256)
        button_classify = gr.Button("Submit to Classify Image", visible = True, size ='sm')


      with gr.Column():
        with gr.Row():
          label_classify = gr.Label(label = "Predicted label", num_top_classes=10, visible = True)

        with gr.Row():
          label_classify_gt = gr.Label(label = "Ground Truth label", visible = True)

    # State to store the source of the selected image
    state = gr.State({"source": None})

    # with gr.Row():
    #     mnist_examples = gr.Examples(get_mnist_images(), inputs=[input_image_classify], label="Select an example MNIST Image")
    #     mnist_examples.change(fn=set_image_source, inputs=[input_image_classify, gr.State(value="MNIST"), state], outputs=state)

    # with gr.Row():
    #     mnist_m_examples = gr.Examples(get_mnist_m_images(), inputs=[input_image_classify], label="Select an example MNIST-M Image")
    #     mnist_m_examples.change(fn=set_image_source, inputs=[input_image_classify, gr.State(value="MNIST-M"), state], outputs=state)

    mnist_images = get_mnist_images()
    mnist_m_images = get_mnist_m_images()
    with gr.Row():
      # gr.Examples(get_mnist_images(),inputs=[input_image_classify], label = "Select an example MNIST Image")
      gr.Examples(mnist_images,inputs=[input_image_classify], label = "Select an example MNIST Image")


    with gr.Row():
      # gr.Examples(get_mnist_m_images(),inputs=[input_image_classify], label = "Select an example MNIST-M Image")
      gr.Examples(mnist_m_images,inputs=[input_image_classify], label = "Select an example MNIST-M Image")

    # Label to display the source of the selected image
    image_source_label = gr.Label(label="Selected Image Source")

    # def update_state(image):
    #     if image in get_mnist_images():
    #         return "MNIST"
    #     elif image in get_mnist_m_images():
    #         return "MNIST-M"
    #     else:
    #         return "Unknown"

    # input_image_classify.change(fn=update_state, inputs=input_image_classify, outputs=image_source_label)
  ##########################
  with gr.Tab("Introduction"):
    gr.Markdown("## Domain Adaptation in Deep Networks - Demonstration")
    gr.Markdown(
        '''
        Source - MNIST
        ------
        - The MNIST database (Modified National Institute of Standards and Technology database) is a large collection of handwritten digits.
        - It has a training set of 60,000 examples, and a test set of 10,000 examples.
        - 28 x 28 size
        - 1 channel

        '''
        )
    gr.Markdown(
        '''
        Target - MNIST-M
        -------
        - MNIST-M is created by combining MNIST digits with the patches randomly extracted from color photos of BSDS500 as their background.
        - It contains 59,001 training and 90,001 test images.
        - 28 x 28 size
        - 3 channels
        '''
    )
    with gr.Row():
        with gr.Column():
             image_output = gr.Image(value=display_image(), label = "source and target",height = 512, width = 512, show_label = True)
  ################

  # button_classify.click(fn=classify_image, inputs =[input_image_classify,slider_classify_num_classes,checkbox_gradcam_classify,dropdown_gradcam_classify_layer,slider_gradcam_classify_opacity], outputs = [label_classify,gallery_gradcam_classify,gallery_gradcam_classify])
  # button_classify.click(fn=classify_image, inputs=[input_image_classify,radio_model,radio_dataset], outputs=label_classify) ## working
  button_classify.click(fn=classify_image,
                        inputs=[input_image_classify,radio_model,gr.State(mnist_images), gr.State(mnist_m_images)],
                        outputs=[label_classify,image_source_label])

  # radio_gradcam.change(fn=view_gradcam_images, inputs=radio_gradcam, outputs=[slider_gradcam_num_images, dropdown_gradcam_layer,slider_gradcam_opacity,button_gradcam, output_gallery_gradcam])



demo.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1001: UserWarning: Expected 1 arguments for function <function classify_image at 0x7e9c69984310>, received 4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1009: UserWarning: Expected maximum 1 arguments for function <function classify_image at 0x7e9c69984310>, received 4.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a9d8c9612ce51430db.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://a9d8c9612ce51430db.gradio.live


In [ ]:

# ### OLD VERSION V1
# with gr.Blocks() as demo:

#   with gr.Tab("MNIST-M : inference NonDANN and DANN"):
#     with gr.Row():
#       radio_gradcam = gr.Radio(["Baseline (Non-DANN)", "DANN"],
#                                label="Select the model you want to use.",
#                                value="Baseline (Non-DANN)",  # Set default value
#                                scale=2)

#     with gr.Row():
#       with gr.Column():
#         input_image_classify = gr.Image(label="Classify Digit", type = "pil", height = 256, width = 256)
#         button_classify = gr.Button("Submit to Classify Image", visible = True)



#       with gr.Column():
#         label_classify = gr.Label(num_top_classes=10, visible = True)

#     with gr.Row():
#       # gr.Markdown("Select the Image")
#       for batch in mnist_loader:
#         images, labels = batch[0],batch[1]
#         image_pils = [display_transforms(img) for img in images]
#         # print(labels)
#         # for img,label in zip (images, labels):
#         #   img_pil = display_transforms(img)
#         #   # with gr.Column():
#         #   gr.Examples([img_pil],inputs=[input_image_classify],label=label.item())
#         # break
#         gr.Examples(image_pils,inputs=[input_image_classify], label = "Select an example MNIST Image")
#         break





#   ##########################
#   with gr.Tab("Introduction"):
#     gr.Markdown("## Domain Adaptation in Deep Networks - Demonstration")
#     gr.Markdown(
#         '''
#         Source - MNIST
#         ------
#         - The MNIST database (Modified National Institute of Standards and Technology database) is a large collection of handwritten digits.
#         - It has a training set of 60,000 examples, and a test set of 10,000 examples.
#         - 28 x 28 size
#         - 1 channel

#         '''
#         )
#     gr.Markdown(
#         '''
#         Target - MNIST-M
#         -------
#         - MNIST-M is created by combining MNIST digits with the patches randomly extracted from color photos of BSDS500 as their background.
#         - It contains 59,001 training and 90,001 test images.
#         - 28 x 28 size
#         - 3 channels
#         '''
#     )
#     with gr.Row():
#         with gr.Column():
#              image_output = gr.Image(value=display_image(), label = "source and target",height = 512, width = 512, show_label = True)
#   ################

#   # button_classify.click(fn=classify_image, inputs =[input_image_classify,slider_classify_num_classes,checkbox_gradcam_classify,dropdown_gradcam_classify_layer,slider_gradcam_classify_opacity], outputs = [label_classify,gallery_gradcam_classify,gallery_gradcam_classify])


#   # radio_gradcam.change(fn=view_gradcam_images, inputs=radio_gradcam, outputs=[slider_gradcam_num_images, dropdown_gradcam_layer,slider_gradcam_opacity,button_gradcam, output_gallery_gradcam])



# demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3c10fcb03a7ecf6937.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Gradio interface setup
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            examples = gr.Examples(
                examples=get_mnist_images(),
                inputs=[gr.Image(type="pil")],
                outputs=[gr.Label(num_top_classes=10, visible=True)],
                label="Select an image to classify"
            )
            input_image = gr.Image(label="Classify Digit", type="pil")
            classify_button = gr.Button("Classify")
            output_label = gr.Label(num_top_classes=10, visible=True)

        def classify_image(image):
            prediction = classify_digit(image)
            return prediction

        classify_button.click(classify_image, inputs=input_image, outputs=output_label)

    with gr.Tab("Introduction"):
        gr.Markdown("## Domain Adaptation in Deep Networks - Demonstration")
        gr.Markdown(
            '''
            Source - MNIST
            ------
            - The MNIST database (Modified National Institute of Standards and Technology database) is a large collection of handwritten digits.
            - It has a training set of 60,000 examples, and a test set of 10,000 examples.
            - 28 x 28 size
            - 1 channel

            '''
        )
        gr.Markdown(
            '''
            Target - MNIST-M
            -------
            - MNIST-M is created by combining MNIST digits with the patches randomly extracted from color photos of BSDS500 as their background.
            - It contains 59,001 training and 90,001 test images.
            - 28 x 28 size
            - 3 channels
            '''
        )

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://def602e9e6dc2c3bba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# import gradio as gr

# def display_image(image):
#     return image

# iface = gr.Interface(
#     fn=display_image,
#     inputs=gr.inputs.Image(type="pil"),
#     outputs=gr.outputs.Image(type="pil"),
#     title="Image Display",
#     description="Upload an image to display it."
# )

# iface.launch()


In [ ]:
# Launch the Gradio app
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0d6269534cc6fe1ba0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import random
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import gradio as gr

# Dummy dataset and dataloader for demonstration
class CustomDataset(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        # Assuming the dataset consists of images and labels
        self.data = [
            ('/content/mnist-m.JPG', 0)
        ]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label, img_path

# Define transforms
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

# Initialize dataset and dataloader
dataset = CustomDataset(transform=transform)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Function to get 10 random images
def get_random_images():
    data_iter = iter(dataloader)
    images, labels, paths = next(data_iter)
    images = [transforms.ToPILImage()(img) for img in images]
    return images, paths

# Dummy model for demonstration
def classify_digit(image):
    # Replace this with actual model inference code
    return "Predicted Label"

# Initialize 10 random images
images, paths = get_random_images()

# Create Gradio interface
def display_images(image_path):
    # Find the corresponding image from paths
    selected_image = None
    for img, path in zip(images, paths):
        if path == image_path:
            selected_image = img
            break

    if selected_image:
        return selected_image, classify_digit(selected_image)

with gr.Blocks() as demo:
    gr.Markdown("## Select an Image and Classify Digit")

    with gr.Row():
        # Display 10 random images
        for img, path in zip(images, paths):
            with gr.Column():
                gr.Image(value=img, label=path, interactive=True).click(
                    display_images, inputs=path, outputs=["image_output", "label_classify"]
                )

    with gr.Row():
        image_output = gr.Image(label="Selected Image")
        label_classify = gr.Label(label="Classification Result")

# Launch the Gradio app
demo.launch()


AttributeError: 'Image' object has no attribute 'click'

In [31]:
import gradio as gr
from PIL import Image

# Define the function to be called with the examples
def classify_image(image):
    return "Label for the image"

# Define the examples and their labels
examples = [
    ("example1.jpg", "Example 1"),
    ("example2.jpg", "Example 2"),
    ("example3.jpg", "Example 3")
]

# Create the Gradio Blocks interface
with gr.Blocks() as demo:
    with gr.Row():
        # Input component
        input_image = gr.Image(type="pil", label="Input Image")

        # Output component
        output_label = gr.Label(label="Output Label")

    # Examples component with custom layout
    with gr.Row():
        for img_path, img_label in examples:
            with gr.Column():
                gr.Image(value=img_path, label=img_label)
                gr.Markdown(img_label)

    # Button to submit the input
    submit_button = gr.Button("Submit")

    # Define the interaction
    submit_button.click(classify_image, inputs=input_image, outputs=output_label)

# Launch the interface
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c01faec8a4fda3d7c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
